# 전사 철자 오류 해결: 프롬프트와 사후 처리 비교하기

특히 회사 이름과 제품 참조와 관련하여 트랜스크립션의 정확성을 높이는 문제를 해결하고 있습니다. 당사의 솔루션에는 속삭임 프롬프트 매개변수와 GPT-4의 후처리 기능을 모두 활용하는 이중 전략이 포함됩니다.

부정확성을 수정하는 두 가지 접근 방식이 있습니다:

- 정확한 철자 목록을 Whisper의 프롬프트 매개변수에 직접 입력하여 초기 트랜스크립션을 안내합니다.

- 전사 후 철자가 틀린 부분을 수정하기 위해 GPT-4를 활용했으며, 프롬프트에 동일한 철자 목록을 다시 사용했습니다.

이러한 전략은 익숙하지 않은 고유명사의 정확한 필사를 보장하기 위한 것이었습니다.

## 설정

시작하려면

- OpenAI Python 라이브러리를 가져옵니다(라이브러리가 없는 경우, '``pip install openai```로 설치해야 합니다).
- 오디오 파일 예제 다운로드

In [1]:
# imports
import openai  # for making OpenAI API calls
import urllib  # for downloading example audio files


In [2]:
# set download paths
ZyntriQix_remote_filepath = "https://cdn.openai.com/API/examples/data/ZyntriQix.wav"


# set local save locations
ZyntriQix_filepath = "data/ZyntriQix.wav"

# download example audio files and save locally
urllib.request.urlretrieve(ZyntriQix_remote_filepath, ZyntriQix_filepath)


('data/ZyntriQix.wav', <http.client.HTTPMessage at 0x11ac6a2b0>)

## 가상의 오디오 녹음으로 기준선 설정하기

우리의 기준점은 작성자가 제공한 프롬프트에서 ChatGPT가 생성한 독백입니다. 그런 다음 작성자가 이 콘텐츠의 음성을 녹음했습니다. 즉, 작성자는 프롬프트를 통해 ChatGPT의 출력을 안내하고 직접 말함으로써 생동감을 불어넣은 것입니다.

가상의 회사 진트리킥스는 다양한 기술 제품을 제공합니다. 여기에는 디지크 플러스, 사이냅스파이브, 보티코어 V8, 에코닉스 어레이, 오비탈링크 세븐, 디지프랙탈 매트릭스 등이 포함됩니다. 또한 PULSE, RAPT, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T. 등 여러 이니셔티브를 주도하고 있습니다.

In [3]:
# define a wrapper function for seeing how prompts affect transcriptions
def transcribe(prompt: str, audio_filepath) -> str:
    """Given a prompt, transcribe the audio file."""
    transcript = openai.Audio.transcribe(
        file=open(audio_filepath, "rb"),
        model="whisper-1",
        prompt=prompt,
    )
    return transcript["text"]


In [4]:
# baseline transcription with no prompt
transcribe(prompt="", audio_filepath=ZyntriQix_filepath)

"Have you heard of ZentricX? This tech giant boasts products like Digi-Q+, Synapse 5, VortiCore V8, Echo Nix Array, and not to forget the latest Orbital Link 7 and Digifractal Matrix. Their innovation arsenal also includes the Pulse framework, Wrapped system, they've developed a brick infrastructure court system, and launched the Flint initiative, all highlighting their commitment to relentless innovation. ZentricX, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

Whisper가 회사 이름과 제품 이름을 잘못 표기하고 약어를 대소문자로 잘못 표기했습니다. 프롬프트에 올바른 이름을 목록으로 전달해 보겠습니다.

In [5]:
# add the correct spelling names to the prompt
transcribe(
    prompt="ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T.",
    audio_filepath=ZyntriQix_filepath,
)


"Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system. They've developed a B.R.I.C.K. infrastructure, Q.U.A.R.T. system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix in just 30 years has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

제품 이름 목록을 전달할 때 일부 제품 이름은 올바르게 입력되지만 다른 제품 이름은 여전히 철자가 틀린 경우가 있습니다.

In [6]:
# add a full product list to the prompt
transcribe(
    prompt="ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T.",
    audio_filepath=ZyntriQix_filepath,
)


"Have you heard of ZentricX? This tech giant boasts products like DigiCube Plus, Synapse 5, VortiCore V8, EchoNix Array, and not to forget the latest Orbital Link 7 and Digifractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system. They've developed a brick infrastructure court system and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZentricX in just 30 years has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?"

## GPT-4를 사용하여 맞춤법 오류를 수정할 수 있습니다.

GPT-4를 활용하면 음성 내용을 미리 알 수 없고 제품 이름 목록이 준비되어 있는 경우에 특히 유용합니다.

GPT-4를 사용한 후처리 기술은 토큰 제한이 244인 Whisper의 프롬프트 매개변수에만 의존하는 것보다 훨씬 더 확장성이 뛰어납니다. GPT-4를 사용하면 정확한 철자가 포함된 더 큰 목록을 처리할 수 있으므로 광범위한 제품 목록을 처리하는 데 더욱 강력한 방법입니다.

하지만 이 후처리 기법에도 한계가 있습니다. 선택한 모델의 컨텍스트 창에 의해 제약을 받기 때문에 방대한 수의 고유 용어를 처리할 때 문제가 발생할 수 있습니다. 예를 들어, 수천 개의 SKU를 보유한 기업은 GPT-4의 컨텍스트 창이 요구 사항을 처리하기에 충분하지 않다는 것을 알게 될 수 있으며, 대체 솔루션을 모색해야 할 수도 있습니다.

흥미로운 점은 GPT-4 후처리 기법이 Whisper를 단독으로 사용하는 것보다 더 안정적이라는 점입니다. 제품 목록을 활용하는 이 방법은 결과의 신뢰성을 높여줍니다. 하지만 이 방식을 사용하면 비용이 증가하고 지연 시간이 길어질 수 있으므로 신뢰성이 높아지는 대신 대가가 따릅니다.

In [7]:
# define a wrapper function for seeing how prompts affect transcriptions
def transcribe_with_spellcheck(system_message, audio_filepath):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": transcribe(prompt="", audio_filepath=audio_filepath),
            },
        ],
    )
    return response["choices"][0]["message"]["content"]


이제 원본 제품 목록을 GPT-4에 입력하고 성능을 평가해 보겠습니다. 이를 통해 트랜스크립션에 표시될 정확한 용어에 대한 사전 지식이 없더라도 AI 모델이 독점 제품 이름의 철자를 정확하게 입력할 수 있는지 평가하고자 합니다. 실험 결과, GPT-4는 제품 이름의 철자를 정확하게 입력하는 데 성공하여 전사 정확도를 보장하는 신뢰할 수 있는 도구로서의 잠재력을 확인했습니다.

In [8]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your task is to correct any spelling discrepancies in the transcribed text. Make sure that the names of the following products are spelled correctly: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T."
new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)
print(new_text)


Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?


이 경우 이전에 사용된 모든 철자와 새로운 이름을 추가로 포함하는 포괄적인 제품 목록을 제공했습니다. 이 시나리오는 상당한 양의 SKU 목록이 있고 트랜스크립션에 표시할 정확한 용어가 불확실한 실제 상황을 시뮬레이션한 것입니다. 이 방대한 제품명 목록을 시스템에 입력한 결과 정확한 필사본이 출력되었습니다.

In [9]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your task is to correct any spelling discrepancies in the transcribed text. Make sure that the names of the following products are spelled correctly: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array,  OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, GigaLink TwentySeven, FusionMatrix TwentyEight, InfiniFractal TwentyNine, MetaSync Thirty, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided."
new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)
print(new_text)


Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just 30 years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?


이전에 프롬프트에 사용되었던 것과 동일한 올바른 철자 목록을 사용하여 맞춤법 검사기로 GPT-4를 사용하고 있습니다.

In [10]:
system_prompt = "You are a helpful assistant for the company ZyntriQix. Your first task is to list the words that are not spelled correctly according to the list provided to you and to tell me the number of misspelled words. Your next task is to insert those correct words in place of the misspelled ones. List: ZyntriQix, Digique Plus, CynapseFive, VortiQore V8, EchoNix Array,  OrbitalLink Seven, DigiFractal Matrix, PULSE, RAPT, AstroPixel Array, QuantumFlare Five, CyberPulse Six, VortexDrive Matrix, PhotonLink Ten, TriCircuit Array, PentaSync Seven, UltraWave Eight, QuantumVertex Nine, HyperHelix X, DigiSpiral Z, PentaQuark Eleven, TetraCube Twelve, GigaPhase Thirteen, EchoNeuron Fourteen, FusionPulse V15, MetaQuark Sixteen, InfiniCircuit Seventeen, TeraPulse Eighteen, ExoMatrix Nineteen, OrbiSync Twenty, QuantumHelix TwentyOne, NanoPhase TwentyTwo, TeraFractal TwentyThree, PentaHelix TwentyFour, ExoCircuit TwentyFive, HyperQuark TwentySix, GigaLink TwentySeven, FusionMatrix TwentyEight, InfiniFractal TwentyNine, MetaSync Thirty, B.R.I.C.K., Q.U.A.R.T.Z., F.L.I.N.T."
new_text = transcribe_with_spellcheck(system_prompt, audio_filepath=ZyntriQix_filepath)
print(new_text)


The misspelled words are: ZentricX, Digi-Q+, Synapse 5, VortiCore V8, Echo Nix Array, Orbital Link 7, Digifractal Matrix, Pulse, Wrapped, brick, Flint, and 30. The total number of misspelled words is 12.

The corrected paragraph is:

Have you heard of ZyntriQix? This tech giant boasts products like Digique Plus, CynapseFive, VortiQore V8, EchoNix Array, and not to forget the latest OrbitalLink Seven and DigiFractal Matrix. Their innovation arsenal also includes the PULSE framework, RAPT system, they've developed a B.R.I.C.K. infrastructure court system, and launched the F.L.I.N.T. initiative, all highlighting their commitment to relentless innovation. ZyntriQix, in just MetaSync Thirty years, has soared from a startup to a tech titan, serving us tech marvels alongside a stimulating linguistic challenge. Quite an adventure, wouldn't you agree?
